In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
cd /content/all_data

/content/all_data


In [ ]:
!kaggle datasets download andrewmvd/covid19-cough-audio-classification

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/covid19-cough-audio-classification
License(s): Attribution 4.0 International (CC BY 4.0)
 98% 1.22G/1.24G [00:21<00:00, 79.1MB/s]
100% 1.24G/1.24G [00:21<00:00, 63.0MB/s]


In [ ]:
! unzip covid19-cough-audio-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: e8e7a8fc-a199-459b-8b47-96aae07cafdc.webm  
  inflating: e8eaf768-9997-4a63-ad2e-4a01c5adee86.json  
  inflating: e8eaf768-9997-4a63-ad2e-4a01c5adee86.webm  
  inflating: e8ebf8aa-107c-4f94-b24b-526e4f5f1b64.json  
  inflating: e8ebf8aa-107c-4f94-b24b-526e4f5f1b64.ogg  
  inflating: e8f0479e-9e75-41e7-806d-dfad3b86ac82.json  
  inflating: e8f0479e-9e75-41e7-806d-dfad3b86ac82.webm  
  inflating: e8f243e4-d456-4b40-bf1e-b6bab899453f.json  
  inflating: e8f243e4-d456-4b40-bf1e-b6bab899453f.webm  
  inflating: e8f45f08-6e1d-4c06-ae96-ce9fe43c91d1.json  
  inflating: e8f45f08-6e1d-4c06-ae96-ce9fe43c91d1.webm  
  inflating: e8f4b378-f21c-40a6-add1-2831323f544e.json  
  inflating: e8f4b378-f21c-40a6-add1-2831323f544e.webm  
  inflating: e8f5bcaa-487d-4e86-93a0-a89c9f87ae93.json  
  inflating: e8f5bcaa-487d-4e86-93a0-a89c9f87ae93.webm  
  inflating: e8f6c938-a196-4e42-9007-a173e99d3700.json  
  inflating: e8f6c938-a196-4e42-9007-a

script for processing json files

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

DATA_DIR = "/content/all_data"  # Change this to your dataset folder
OUTPUT_DIR = "/content/"  # Destination folder in Google Drive
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "jsondata.csv")  # Full path to save the file

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

import random

def preprocess_data(data_list, all_keys, filenames):
    """Process JSON data, handle missing values, and prepare structured data."""
    df = pd.DataFrame(data_list)

    # Add 'filename' column as the last column
    df['filename'] = filenames

    # Fill missing values for keys that don't exist in some files
    for key in all_keys:
        if key not in df:
            df[key] = np.nan  # Add missing keys as NaN

    # Convert numerical values
    numerical_features = ["cough_detected", "age", "latitude", "longitude"]
    for col in numerical_features:
        if col in df:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col].fillna(df[col].mean(), inplace=True)  # Fill missing values with mean

    # Convert categorical values
    categorical_features = ["gender", "status", "respiratory_condition", "fever_muscle_pain"]
    for col in categorical_features:
        if col in df:
            df[col] = df[col].astype(str).fillna("unknown")

    # Flatten nested dictionaries like "expert_labels_4"
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

    # Normalize numerical data
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    return df

if __name__ == "__main__":
    data_list, all_keys, filenames, selected_filenames = load_json_data(DATA_DIR)

    processed_df = preprocess_data(data_list, all_keys, filenames)

    # Save processed data to Google Drive
    processed_df.to_csv(OUTPUT_FILE, index=False)
    print(f"✅ Data processing complete. Saved as '{OUTPUT_FILE}'.")


Processing 2755 JSON files out of 27550 (10% sample).
✅ Data processing complete. Saved as '/content/jsondata.csv'.


<ipython-input-8-81c5d994367a>:59: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)  # Fill missing values with mean


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import subprocess
import multiprocessing

try:
    from google.colab import files  # Import files module for auto-download in Google Colab
    colab_env = True
except ImportError:
    colab_env = False  # If not running in Colab, handle downloads differently

# Directories
AUDIO_DIR = "/content/all_data"  # Change this to your audio dataset folder
OUTPUT_DIR = "/content/"  # Destination folder for CSV file
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "audiodata.csv")  # Full path for output CSV
TEMP_WAV_DIR = os.path.join(OUTPUT_DIR, "tempw")  # Folder for converted audio files

# Ensure the output and temp directories exist
os.makedirs(TEMP_WAV_DIR, exist_ok=True)

# Feature column names
feature_columns = [f"mfcc_{i}" for i in range(8)] + \
                  [f"spec_contrast_{i}" for i in range(7)]

def check_ffmpeg():
    """Ensure FFmpeg is installed."""
    try:
        result = subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if "ffmpeg version" in result.stdout:
            print("✅ FFmpeg is installed.")
        else:
            print("⚠️ FFmpeg is missing. Installing...")
            subprocess.run(["apt-get", "install", "ffmpeg"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except FileNotFoundError:
        print("❌ FFmpeg is not installed. Install it before running this script.")

def batch_convert_to_wav(audio_dir, selected_filenames):
    """Converts only selected OGG/WEBM files to WAV using FFmpeg."""
    for filename in selected_filenames:
        for ext in [".ogg", ".webm"]:
            input_path = os.path.join(audio_dir, filename + ext)
            output_path = os.path.join(TEMP_WAV_DIR, filename + ".wav")

            if os.path.exists(input_path) and not os.path.exists(output_path):  # Convert only if not already converted
                subprocess.run(["ffmpeg", "-i", input_path, "-ar", "22050", "-ac", "1", output_path, "-y"],
                               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def extract_features(file_path):
    """Extracts simplified audio features."""
    try:
        y, sr = librosa.load(file_path, sr=22050, duration=5)  # Load only 5 sec

        if len(y) == 0:  # Skip silent or empty audio
            print(f"⚠️ Warning: {file_path} is silent. Skipping.")
            return None

        # Extract features
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=8)
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

        feature_vector = np.hstack([np.mean(mfccs, axis=1), np.mean(spectral_contrast, axis=1)])

        return feature_vector
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_single_file(filename):
    """Processes a single file and extracts features."""
    file_path = os.path.join(TEMP_WAV_DIR, filename)

    if file_path.endswith(".wav"):
        features = extract_features(file_path)
        if features is not None:
            return filename, features
    return None

def process_audio_files(selected_filenames):
    """Processes only the selected 10% of audio files that match JSON selections."""

    print(f"🎵 Processing {len(selected_filenames)} audio files...")

    # Convert selected OGG/WEBM files to WAV
    batch_convert_to_wav(AUDIO_DIR, selected_filenames)

    # Process only selected WAV files
    filenames = [f for f in os.listdir(TEMP_WAV_DIR) if f.endswith(".wav") and f.split('.')[0] in selected_filenames]

    # Debugging: Check if any files exist
    if not filenames:
        print("⚠️ No WAV files found for processing. Check your audio directory and selection logic.")
        return

    print(f"🔍 Found {len(filenames)} WAV files for feature extraction.")

    # Parallel processing
    with multiprocessing.Pool(processes=min(4, multiprocessing.cpu_count())) as pool:  # Use up to 4 cores
        results = pool.map(process_single_file, filenames)

    # Filter out None values and save results
    results = [r for r in results if r is not None]

    if results:
        new_filenames, new_features = zip(*results)
        df_new = pd.DataFrame(new_features, columns=feature_columns)
        df_new["filename"] = new_filenames

        # Append to CSV
        df_new.to_csv(OUTPUT_CSV, mode="a", header=not os.path.exists(OUTPUT_CSV), index=False)

        print(f"✅ Audio processing complete. Saved as '{OUTPUT_CSV}'.")
    else:
        print("❌ No valid features extracted. Check for corrupt audio files.")


In [ ]:
check_ffmpeg()
process_audio_files(selected_filenames)  # Make sure selected_filenames is correctly populated


✅ FFmpeg is installed.
🎵 Processing 2755 audio files...
🔍 Found 2755 WAV files for feature extraction.
✅ Audio processing complete. Saved as '/content/audiodata.csv'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import joblib
# Load the CSV files
jsondata = pd.read_csv('/content/jsondata.csv')
audiodata = pd.read_csv('/content/audiodata.csv')

# Remove the file extensions by splitting the filename at '.' and keeping the first part
jsondata['filename'] = jsondata['filename'].apply(lambda x: x.split('.')[0])
audiodata['filename'] = audiodata['filename'].apply(lambda x: x.split('.')[0])

# Merge the two dataframes on the 'filename' column
merged_data = pd.merge(jsondata, audiodata, on='filename', how='outer')
# Drop unnecessary columns
columns_to_drop = ['datetime','latitude', 'longitude',
                   'expert_labels_1', 'expert_labels_2', 'expert_labels_3', 'expert_labels_4']
merged_data = merged_data.drop(columns=columns_to_drop)

# Display the merged data
print(merged_data.head())

# Handle missing values (drop or fill)
merged_data = merged_data.dropna(subset=['filename'])  # Drop rows where filenames are missing
# Drop rows where the target variable (y) has NaN values
merged_data = merged_data.dropna(subset=['cough_detected'])

# Define the target variable (you need to define the actual target column here)
# Separate numerical and categorical columns
numerical_features = ['age', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
                      'spec_contrast_0', 'spec_contrast_1', 'spec_contrast_2', 'spec_contrast_3', 'spec_contrast_4',
                      'spec_contrast_5', 'spec_contrast_6']
categorical_features = ['gender_male', 'gender_nan', 'gender_other', 'status_healthy', 'status_nan', 'status_symptomatic',
                        'respiratory_condition_True', 'respiratory_condition_nan', 'fever_muscle_pain_True', 'fever_muscle_pain_nan']

# Define the target variable
y = merged_data['cough_detected']  # Ensure this is the correct target column

# Select relevant features
X = merged_data[numerical_features + categorical_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to preprocess the features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Standardize numerical features
        ('cat', OneHotEncoder(drop='first'), categorical_features)  # One-hot encode categorical features
    ])

# Create the model pipeline that first applies preprocessing and then trains the regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

# Save the model pipeline for later use
joblib.dump(pipeline, '/content/breathe_model_pipeline.pkl')

   cough_detected       age                              filename  \
0        0.818032  0.000000  002db0bd-e57f-4c30-ade0-16640d424eb7   
1        0.446456  0.175167  0037f67c-0d6f-42e6-ab94-66499d4d2bf4   
2        0.908126  0.000000  003c7941-a3ef-4daf-b8a8-5c77c82504b0   
3        0.899794  0.000000  00420d07-89f8-464d-88dd-8e893f0c3f56   
4       -0.551356  0.000000  00430a83-0fd3-4e5c-ae81-82a541f43d3a   

   gender_male  gender_nan  gender_other  status_healthy  status_nan  \
0        False        True         False           False        True   
1        False       False          True            True       False   
2        False        True         False           False        True   
3        False        True         False           False        True   
4        False        True         False           False        True   

   status_symptomatic  respiratory_condition_True  ...     mfcc_5     mfcc_6  \
0               False                       False  ...   6.907457 -13.20

['/content/breathe_model_pipeline.pkl']

In [ ]:
import pandas as pd

# Load the CSV files
jsondata = pd.read_csv('/content/jsondata.csv')
audiodata = pd.read_csv('/content/audiodata.csv')

# Remove the file extensions by splitting the filename at '.' and keeping the first part
jsondata['filename'] = jsondata['filename'].apply(lambda x: x.split('.')[0])
audiodata['filename'] = audiodata['filename'].apply(lambda x: x.split('.')[0])

# Merge the two dataframes on the 'filename' column
merged_data = pd.merge(jsondata, audiodata, on='filename', how='outer')
# Drop unnecessary columns
columns_to_drop = ['datetime','latitude', 'longitude',
                   'expert_labels_1', 'expert_labels_2', 'expert_labels_3', 'expert_labels_4']
merged_data = merged_data.drop(columns=columns_to_drop)

# Display the merged data
print(merged_data.head())

# Handle missing values (drop or fill)
merged_data = merged_data.dropna(subset=['filename'])  # Drop rows where filenames are missing
merged_data = merged_data.dropna(subset=['cough_detected'])  # Drop rows where the target variable (cough_detected) has NaN values

# Define the target variable
y = merged_data['cough_detected']  # Ensure this is the correct target column

# Separate numerical and categorical columns
numerical_features = ['age', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
                      'spec_contrast_0', 'spec_contrast_1', 'spec_contrast_2', 'spec_contrast_3', 'spec_contrast_4',
                      'spec_contrast_5', 'spec_contrast_6']
categorical_features = ['gender_male', 'gender_nan', 'gender_other', 'status_healthy', 'status_nan', 'status_symptomatic',
                        'respiratory_condition_True', 'respiratory_condition_nan', 'fever_muscle_pain_True', 'fever_muscle_pain_nan']

# Select relevant features
X = merged_data[numerical_features + categorical_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to preprocess the features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Standardize numerical features
        ('cat', OneHotEncoder(drop='first'), categorical_features)  # One-hot encode categorical features
    ])

# Create the model pipeline with XGBoost
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
])

# Define hyperparameters for GridSearchCV
param_grid = {
    'model__n_estimators': [50, 100, 150],  # Number of boosting rounds
    'model__max_depth': [3, 5, 7],  # Max depth of trees
    'model__learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'model__subsample': [0.8, 1.0],  # Fraction of samples used for training
    'model__colsample_bytree': [0.8, 1.0]  # Fraction of features used per tree
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

# Train the best model from GridSearchCV
best_pipeline = grid_search.best_estimator_

# Evaluate the model
y_pred = best_pipeline.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

# Save the model pipeline for later use
joblib.dump(best_pipeline, '/content/breathe_xgboost_model_pipeline.pkl')

   cough_detected       age                              filename  \
0        0.818032  0.000000  002db0bd-e57f-4c30-ade0-16640d424eb7   
1        0.446456  0.175167  0037f67c-0d6f-42e6-ab94-66499d4d2bf4   
2        0.908126  0.000000  003c7941-a3ef-4daf-b8a8-5c77c82504b0   
3        0.899794  0.000000  00420d07-89f8-464d-88dd-8e893f0c3f56   
4       -0.551356  0.000000  00430a83-0fd3-4e5c-ae81-82a541f43d3a   

   gender_male  gender_nan  gender_other  status_healthy  status_nan  \
0        False        True         False           False        True   
1        False       False          True            True       False   
2        False        True         False           False        True   
3        False        True         False           False        True   
4        False        True         False           False        True   

   status_symptomatic  respiratory_condition_True  ...     mfcc_5     mfcc_6  \
0               False                       False  ...   6.907457 -13.20

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

/content/drive/MyDrive/Breathe/testaudio.m4a

In [ ]:
import librosa
import numpy as np
import pandas as pd
import joblib

# Load the trained model pipeline
pipeline = joblib.load('/content/breathe_model_pipeline.pkl')

# Define the features based on your previous code (numerical and categorical)
numerical_features = ['age', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
                      'spec_contrast_0', 'spec_contrast_1', 'spec_contrast_2', 'spec_contrast_3', 'spec_contrast_4',
                      'spec_contrast_5', 'spec_contrast_6']

categorical_features = ['gender_male', 'gender_nan', 'gender_other', 'status_healthy', 'status_nan', 'status_symptomatic',
                        'respiratory_condition_True', 'respiratory_condition_nan', 'fever_muscle_pain_True', 'fever_muscle_pain_nan']

# Function to extract MFCC and Spectral Contrast from an audio file
def extract_audio_features(audio_path):
    # Load the audio file
    audio, sr = librosa.load(audio_path, sr=None)  # sr=None to preserve original sampling rate

    # Extract MFCC features (first 8 coefficients)
    mfcc = librosa.feature.mfcc(audio, sr=sr, n_mfcc=8)
    mfcc_features = np.mean(mfcc, axis=1)  # Take mean over time for each coefficient

    # Extract Spectral Contrast (first 7 bands)
    spec_contrast = librosa.feature.spectral_contrast(audio, sr=sr)
    spec_contrast_features = np.mean(spec_contrast, axis=1)  # Take mean over time

    # Combine MFCC and Spectral Contrast features into a single feature vector
    features = np.concatenate((mfcc_features, spec_contrast_features))
    return features

# Function to prepare the input for prediction
def prepare_input(audio_features):
    # Default values for categorical features (set these according to your needs)
    default_categorical = {
        'gender_male': 0, 'gender_nan': 0, 'gender_other': 0,
        'status_healthy': 1, 'status_nan': 0, 'status_symptomatic': 0,
        'respiratory_condition_True': 0, 'respiratory_condition_nan': 0,
        'fever_muscle_pain_True': 0, 'fever_muscle_pain_nan': 0
    }

    # Combine numerical and categorical features into a DataFrame
    input_data = pd.DataFrame([audio_features], columns=numerical_features)

    # Add the categorical features as columns in the DataFrame
    for feature in categorical_features:
        input_data[feature] = default_categorical[feature]

    return input_data

# Path to the audio file you want to test
audio_path = 'content/testaudio.m4a'  # Replace with your actual file path

# Extract features from the audio file
audio_features = extract_audio_features(audio_path)

# Prepare the input data (combine audio features with default categorical values)
input_data = prepare_input(audio_features)

# Use the pipeline to make a prediction
prediction = pipeline.predict(input_data)

# Output the prediction (whether cough detected or not)
print("Cough Detected Probability:", prediction[0])
